## Garbage Collection

In [1]:
import ctypes
import gc

In [2]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return 'Object exist'
    return 'Not found'

we define two classes that we will use to create a circular reference


In [4]:
class A:
    def __init__(self):
        self.b = B(self)
        print(f'A: self: {hex(id(self))}, b: {hex(id(self.b))}')

In [5]:
class B:
    def __init__(self, a):
        self.a = a
        print(f'B: self: {hex(id(self))}, a: {hex(id(self.a))}')

We turn off the GC so we can see how reference counts are affected when the GC does not run and when it does (by running it manually).

In [6]:
gc.disable()

Now we create an instance of A, which will, in turn, create an instance of B which will store a reference to the calling A instance.

In [7]:
my_var = A()

B: self: 0x7ff0c2c55b40, a: 0x7ff0c2c55ab0
A: self: 0x7ff0c2c55ab0, b: 0x7ff0c2c55b40


In fact `my_var` is also a reference to the same A instance:

In [8]:
hex(id(my_var))

'0x7ff0c2c55ab0'

In [9]:
print('a: \t{0}'.format(hex(id(my_var))))
print('b: \t{0}'.format(hex(id(my_var.b))))
print('b.a: \t{0}'.format(hex(id(my_var.b.a))))

a: 	0x7ff0c2c55ab0
b: 	0x7ff0c2c55b40
b.a: 	0x7ff0c2c55ab0


In [10]:
a_id = id(my_var)
b_id = id(my_var.b)

In [11]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 2
refcount(b) = 1
a: Object exist
b: Object exist


In [12]:
my_var= None

In [13]:

print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 1
refcount(b) = 1
a: Object exist
b: Object exist


As we can see, the reference counts are now both equal to 1 (a pure circular reference), and reference counting alone did not destroy the A and B instances - they're still around. If no garbage collection is performed this would result in a memory leak.

Let's run the GC manually and re-check whether the objects still exist:

In [14]:
gc.collect()


164

In [15]:
print('refcount(a) = {0}'.format(ref_count(a_id)))
print('refcount(b) = {0}'.format(ref_count(b_id)))
print('a: {0}'.format(object_by_id(a_id)))
print('b: {0}'.format(object_by_id(b_id)))

refcount(a) = 0
refcount(b) = 0
a: Not found
b: Not found
